<div style="background-color: black; color: white; padding: 10px;text-align: center;">
  <strong>Date Published:</strong> June 30, 2025 <strong>Author:</strong> Adnan Alaref
</div>

# 🎯 Goal

The goal of this notebook is to gain a deep understanding of the internal mechanics of **Gaussian Mixture Models (GMMs)** by:

- Generating synthetic **univariate and multivariate** datasets
- Fitting GMMs using `sklearn.mixture.GaussianMixture`
- Implementing custom versions of the `score_samples()` method from scratch for both univariate and multivariate cases
- Validating our implementations by comparing against sklearn’s built-in log-likelihood outputs

# **◍ Step 1: Introduction.**

✅ 1. Gaussian Mixture Model (GMM) Probability Density Function
The probability of a data point x under a GMM is:  
For K components:

$$
p(x) = \sum_{k=1}^{K} \pi_k \cdot \mathcal{N}(x \mid \mu_k, \Sigma_k)
$$

Where:
* πk: mixing coefficient, ∑πₖ=1 , πₖ > 0
* μk: mean of component 𝙺
* Σk: covariance matrix of component 𝙺
* N(x∣μₖ,Σₖ​): multivariate normal density.

---

**The Probability Density Function (PDF) of a multivariate (or univariate) normal distribution.**

✅ 1. Univariate Normal Distribution (1D)  
If x ∈ ℝ, then:
$$
f(x \mid \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \cdot \exp\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)
$$
Where:

* μ is the mean
* σ² is the variance
* σ is the standard deviation

---

✅ 2. Multivariate Normal Distribution (d-dimensional)
If x∈R d, then:

$$
f(\mathbf{x} \mid \boldsymbol{\mu}, \boldsymbol{\Sigma}) =
\frac{1}{(2\pi)^{d/2} \cdot |\boldsymbol{\Sigma}|^{1/2}} \cdot
\exp\left( -\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu})^\top \boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu}) \right)
$$

Where:

* x is a d-dimensional vector
* μ is the mean vector (d,)
* Σ is the covariance matrix (d × d)
* |Σ| is the determinant of the covariance matrix
* Σ⁻¹ is the inverse of the covariance matrix



## **📝 Notes For Code:**
* **`.reval() Method`**
  - Flattens the array to 1D, shape (n_components,)Same as .reshape(-1)
  - Useful to ensure consistent shape for broadcasting, plotting, or element-wise math

* **`.einsum('ij,jk,ik->i', diff, inv_cov, diff)`**
  - This computing the __Mahalanobis distance squared__ for each sample vector in **diff** (X - μ), given the __inverse covariance matrix__ .
  - Let's break it down:    
  - **Given:**

    - \(𝕩\): a sample point (vector)  
    - \(𝛍\): the mean vector of the distribution  
    - \(𝚺\): the covariance matrix  

  - **The Mahalanobis distance is:**

    $$
    D_M(\mathbf{x}) = \sqrt{(\mathbf{x} - \boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu})}
    $$

  - **Squaring it gives:**

  $$
  D_M^2(\mathbf{x}) = (\mathbf{x} - \boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu})
  $$

  - 🔹 Explanation of np.einsum('ij,jk,ik->i', diff, inv_cov, diff):
    - `ij` : rows i of diff (shape (n_samples, n_features))

    - `jk` : inv_cov matrix (shape (n_features, n_features))

    - `ik` : again diff (same as first, reused)   
   ✅ This is what's being computed — but **for many samples at once.**


# **◍ Step 2: Import Librares.**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore' ,category=FutureWarning)

# **◍ Step 3: Generate Synthetic DataSet(Multivariate).**

In [2]:
data, true_labels = make_blobs(n_samples=400, n_features=2 ,centers=4, cluster_std=1.0, random_state=42)
print(f"Dataset Shape : {data.shape} - Ground_Truth shape : {true_labels.shape}\n")
print("Explore 5 Rows From Dataset : \n",data[1:5],"\n")
print("Explore True_Labels: ",true_labels[1:15],"\n")

Dataset Shape : (400, 2) - Ground_Truth shape : (400,)

Explore 5 Rows From Dataset : 
 [[-5.12894273  9.83618863]
 [-9.08407082  7.05079935]
 [ 5.61499857  1.8261123 ]
 [ 5.21076935  3.10873532]] 

Explore True_Labels:  [0 3 1 1 0 0 3 0 1 3 1 3 1 2] 



# **◍ Step 4: Generate Synthetic DataSet(Univariate).**

In [3]:
data1_d,true_labels1_d = make_blobs(n_samples=400, n_features=1, centers=4, cluster_std=1.0, random_state=42)
print(f"Dataset Shape : {data1_d.shape} - Ground_Truth shape : {true_labels1_d.shape}\n")
print("Explore 5 Rows From Dataset : \n",data1_d[1:5],"\n")
print("Explore True_Labels: ",true_labels[1:15],"\n")

Dataset Shape : (400, 1) - Ground_Truth shape : (400,)

Explore 5 Rows From Dataset : 
 [[-2.90130578]
 [ 9.67083974]
 [ 0.97064032]
 [ 9.27416892]] 

Explore True_Labels:  [0 3 1 1 0 0 3 0 1 3 1 3 1 2] 



# **◍ Step 5: Build GMM Model On(Multivariate Dataset).**

In [4]:
MultivariateGMM = GaussianMixture(n_components=4, covariance_type='full', random_state=42)
MultivariateGMM.fit(data)

print(MultivariateGMM)
log_likelihood = MultivariateGMM.score_samples(data)
print(f"Log_likelihood = {log_likelihood[:5]} , Size = {log_likelihood.shape}\n") # (400,)

GaussianMixture(n_components=4, random_state=42)
Log_likelihood = [-5.09812703 -7.67507046 -3.28714688 -3.52994779 -4.04572939] , Size = (400,)



## **🔹 5.1: Output Model Paramters : πk, μk, Σk**

In [5]:
print("Means: ", MultivariateGMM.means_ , " ⟶ Shape: ",MultivariateGMM.means_.shape)
print("\nCovariances: ", MultivariateGMM.covariances_," ⟶ Shape: ",MultivariateGMM.covariances_.shape)
print("\nMixture wheights: ",MultivariateGMM.weights_," ⟶ Shape : ",MultivariateGMM.weights_.shape)

Means:  [[-2.63340741  9.0434803 ]
 [-6.88387179 -6.98398415]
 [ 4.74710337  2.01059427]
 [-8.92932115  7.38197498]]  ⟶ Shape:  (4, 2)

Covariances:  [[[ 0.7493203   0.04400541]
  [ 0.04400541  0.9872469 ]]

 [[ 1.02961821  0.09574043]
  [ 0.09574043  0.98277206]]

 [[ 1.07033552 -0.09844297]
  [-0.09844297  0.85003382]]

 [[ 0.98717804 -0.14616473]
  [-0.14616473  1.00927728]]]  ⟶ Shape:  (4, 2, 2)

Mixture wheights:  [0.25001131 0.25       0.25       0.24998869]  ⟶ Shape :  (4,)


## **🔹 5.2: Custom Implemintation To Score_sample For Multivariate Data In GMM.**
* **The Formula:**
$$
f(\mathbf{x} \mid \boldsymbol{\mu}, \boldsymbol{\Sigma}) =
\frac{1}{(2\pi)^{d/2} \cdot |\boldsymbol{\Sigma}|^{1/2}} \cdot
\exp\left( -\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu})^\top \boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu}) \right)
$$

In [6]:
def our_score_samples_multivariate(gmm, X):
  n_samples, n_features = X.shape
  sum_likelihood = np.zeros(n_samples, dtype=np.float64)

  for k in range(gmm.n_components):
    mu = gmm.means_[k]
    pi = gmm.weights_[k]
    cov = gmm.covariances_[k]

    # Add a tiny value for numerical stability
    cov =cov + np.eye(n_features) * 1e-8

    # Pre-compute determinant and inverse
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)

    # Compute Mahalanobis distance using einsum
    diff = X - mu                           # shape (n_samples, n_features)
    mdist = np.einsum('ij,jk,ik->i', diff, inv_cov, diff)  # shape (n_samples,)

    # Compute multivariate normal density
    norm_factor = np.sqrt((2 *np.pi) ** n_features * det_cov)
    exp_term = np.exp(-0.5 * mdist)

    # Total weighted likelihood for this component
    probs = pi * (exp_term / norm_factor)

    sum_likelihood +=probs

  # Prevent log(0)
  sum_likelihood = np.maximum(sum_likelihood,1e-300)
  return np.log(sum_likelihood)

## **🔹 5.3: Test Custom Score_sample Method.**

In [7]:
log_likelihood_custom = our_score_samples_multivariate(MultivariateGMM ,data)

# Show insights
print("sklearn:", log_likelihood[:5] ,"Shape : ",log_likelihood.shape)
print("custom :", log_likelihood_custom[:5],"Shape : ",log_likelihood_custom.shape)

# Correct way to compare floating-point arrays
if log_likelihood.shape == log_likelihood_custom.shape:
    if np.allclose(log_likelihood, log_likelihood_custom, atol=1e-5):
        print("Custom implementation matches sklearn's results ✅")
    else:
        print("❌ Not Match?.")
else:
    print("❌ Shape mismatch:", log_likelihood.shape, log_likelihood_custom.shape)

sklearn: [-5.09812703 -7.67507046 -3.28714688 -3.52994779 -4.04572939] Shape :  (400,)
custom : [-5.09812702 -7.67507041 -3.28714689 -3.5299478  -4.04572939] Shape :  (400,)
Custom implementation matches sklearn's results ✅


# **◍ Step 6: Build GMM Model On(Univariate Dataset).**

In [8]:
UnivariateGMM = GaussianMixture(n_components=4, covariance_type='full', random_state=42)
UnivariateGMM.fit(data1_d)

print(UnivariateGMM)
log_likelihood_1d = UnivariateGMM.score_samples(data1_d)
print(f"Log_likelihood = {log_likelihood_1d[:5]} , Size = {log_likelihood_1d.shape}\n") # (400,)

GaussianMixture(n_components=4, random_state=42)
Log_likelihood = [-2.31345486 -2.24354037 -2.4831697  -2.92316033 -2.30920551] , Size = (400,)



## **🔹 6.1: Output Model Paramters : πk, μk, Σk**

In [9]:
print("Means: ", UnivariateGMM.means_ , " ⟶ Shape: ",UnivariateGMM.means_.shape)
print("\nCovariances: ", UnivariateGMM.covariances_," ⟶ Shape: ",UnivariateGMM.covariances_.shape)
print("\nMixture wheights: ",UnivariateGMM.weights_," ⟶ Shape : ",UnivariateGMM.weights_.shape)

Means:  [[-2.65723445]
 [ 9.02407488]
 [ 4.59637423]
 [ 2.0172488 ]]  ⟶ Shape:  (4, 1)

Covariances:  [[[0.82892192]]

 [[1.02251277]]

 [[0.8260163 ]]

 [[0.69948095]]]  ⟶ Shape:  (4, 1, 1)

Mixture wheights:  [0.25095445 0.25961582 0.24316642 0.24626331]  ⟶ Shape :  (4,)


## **🔹 6.2: Custom Implemintation To Score_sample For Univariate Data In GMM.**
* **The Formula:**
$$
f(x \mid \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \cdot \exp\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)
$$

In [10]:
# Custom implementation of score_samples for univariate GMM
def our_score_samples_1d(gmm, x):
    def gaussian_density(x, mu, std):
        return (1 / (np.sqrt(2 * np.pi) * std)) * \
               np.exp(-0.5 * ((x - mu) / std) ** 2)

    sum_likelihood = np.zeros_like(x, dtype=np.float64)

    for mu, variance, pi in zip(gmm.means_.ravel(), gmm.covariances_.ravel(), gmm.weights_):
        std = np.sqrt(max(variance, 1e-8))  # Avoid zero std
        probs = pi * gaussian_density(x, mu, std)
        sum_likelihood += probs

    # Avoid log(0)
    sum_likelihood = np.maximum(sum_likelihood, 1e-300)
    return np.log(sum_likelihood).ravel()

## **🔹 6.3: Test Custom Score_sample Method.**

In [11]:
log_likelihood_1d_custom = our_score_samples_1d(UnivariateGMM ,data1_d)

print("Sklearn:", log_likelihood_1d[:5] ,"Shape : ",log_likelihood_1d.shape)
print("Custom :", log_likelihood_1d_custom[:5],"Shape : ",log_likelihood_1d_custom.shape)

# Correct way to compare floating-point arrays
if log_likelihood_1d.shape == log_likelihood_1d_custom.shape:
    if np.allclose(log_likelihood_1d, log_likelihood_1d_custom, atol=1e-5):
        print("Custom implementation matches sklearn's results ✅")
    else:
        print("❌ Not Match?.")
else:
    print("❌ Shape mismatch:", log_likelihood_1d.shape, log_likelihood_1d_custom.shape)

Sklearn: [-2.31345486 -2.24354037 -2.4831697  -2.92316033 -2.30920551] Shape :  (400,)
Custom : [-2.31345486 -2.24354037 -2.4831697  -2.92316033 -2.30920551] Shape :  (400,)
Custom implementation matches sklearn's results ✅


## ✅ Conclusion

In this notebook, we thoroughly explored Gaussian Mixture Models (GMMs) by implementing and validating core functionality from scratch.

### 🔧 What We Did:
- Built and visualized synthetic **univariate and multivariate datasets**
- Fit models using `sklearn.mixture.GaussianMixture`
- Implemented **custom `score_samples()`** functions for:
  - **Univariate GMMs** using simple Gaussian PDFs
  - **Multivariate GMMs** using full covariance matrices and Mahalanobis distance
- Compared our custom outputs with sklearn’s built-in results and verified their correctness

### 🧠 Key Takeaways:
- GMMs estimate **data likelihood** as a mixture of multiple Gaussian components, weighted by prior probabilities.
- Custom implementation of `score_samples()` builds deep intuition for:
  - Log-likelihood computation
  - Multivariate density via Mahalanobis distance
  - Numerical stability (e.g., `log(1e-300)` instead of `log(0)`)
- Matching sklearn's results demonstrates a strong understanding of **probabilistic models**, and how they differ from hard clustering (e.g., k-means).

<a id="Import"></a>
<p style="background-color: #000000; font-family: 'Verdana', sans-serif; color: #FFFFFF; font-size: 160%; text-align: center; border-radius: 25px; padding: 12px 20px; margin-top: 20px; border: 2px solid transparent; background-image: linear-gradient(black, black), linear-gradient(45deg, #FF00FF, #00FFFF, #FFFF00, #FF4500); background-origin: border-box; background-clip: content-box, border-box; box-shadow: 0px 4px 20px rgba(255, 105, 180, 0.8);">
   Thanks & Upvote ❤️</p>